# Click-Through Rate Prediction Model with Machine Learning
---

본 글은 [AMAN KHARWAL님의 글](https://thecleverprogrammer.com/2021/01/24/click-through-rate-prediction-with-machine-learning/)을 보고 실습해본 것이다.

광고 회사는 CTR을 예측하여 광고에 응답할 가능성이 가장 높은 방문자들을 선택하고, 검색 기록을 분석하고 유저의 관심사에 따라 가장 관련된 광고들을 보여준다. 

이 일은 모든 광고 에이전시에게 중요하다. 왜냐하면 인터넷 상의 프로모션들의 상업적 가치는 오직 사용자가 광고에 얼마나 반응하냐에 달려있기 때문이다. 광고에 대한 유저의 반응은 모든 광고 회사에게 매우 가치있다. 왜냐하면, 이것이 회사가 유저에게 가장 관련있는 광고를 선택할 수 있게 해주기 때문이다. 

이제 간단하게 파이썬을 이용한 머신러닝으로 CTR 예측을 위한 모델을 어떻게 만드는지 봐보자. 

먼저 데이터 셋을 불러오자. 저자의 데이터 다운로드 버튼이 작동하지 않아서, 나는 캐글의 [다운로드 링크](https://www.kaggle.com/datasets/gauravduttakiit/clickthrough-rate-prediction?resource=download)에서 따로 받았다. 


In [4]:
import pandas as pd
data = pd.read_csv('archive/ad_10000records.csv')
data.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad
0,62.26,32.0,69481.85,172.83,Decentralized real-time circuit,Lisafort,Male,Svalbard & Jan Mayen Islands,2016-06-09 21:43:05,0
1,41.73,31.0,61840.26,207.17,Optional full-range projection,West Angelabury,Male,Singapore,2016-01-16 17:56:05,0
2,44.40,30.0,57877.15,172.83,Total 5thgeneration standardization,Reyesfurt,Female,Guadeloupe,2016-06-29 10:50:45,0
3,59.88,28.0,56180.93,207.17,Balanced empowering success,New Michael,Female,Zambia,2016-06-21 14:32:32,0
4,49.21,30.0,54324.73,201.58,Total 5thgeneration standardization,West Richard,Female,Qatar,2016-07-21 10:54:35,1


null 값이 있는지 확인하자. isnull()을 하면 모든 데이터들에 대해 null인지 아닌지를 파악해준다. 그리고 False는 0이기 때문에, sum()을 이용해서 null 값의 개수를 확인할 수 있다.

In [7]:
data.isnull().sum() 

Daily Time Spent on Site    0
Age                         0
Area Income                 0
Daily Internet Usage        0
Ad Topic Line               0
City                        0
Gender                      0
Country                     0
Timestamp                   0
Clicked on Ad               0
dtype: int64

null 값이 하나도 존재하지 않는다! 

이제 머신러닝 모델에 적용하기 위해서, 다음의 필요한 feature들만 남기고 나머지는 제거하자. 

* Daily Time Spent on Site	
* Age	
* Area Income	
* Daily Internet Usage	
* Gender

그리고 Gender에서 Male을 0으로, Female을 1로 바꿔주자.

In [24]:
x = data.iloc[:,0:7]
x = x.drop(['Ad Topic Line','City'],axis=1)
x = x.replace({'Gender' : 'Male'}, 0)
x = x.replace({'Gender' : 'Female'}, 1)
x.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Gender
0,62.26,32.0,69481.85,172.83,0
1,41.73,31.0,61840.26,207.17,0
2,44.40,30.0,57877.15,172.83,1
3,59.88,28.0,56180.93,207.17,1
4,49.21,30.0,54324.73,201.58,1


target으로 **"Clicked on Ad"**를 사용하자. 광고를 클릭 했는지(True) 안했는지(False)에 대한 bool인것 같다.

In [25]:
y = data.iloc[:,9]
y.head()

0    0
1    0
2    0
3    0
4    1
Name: Clicked on Ad, dtype: int64

이제 학습과 테스트를 위해서 7:3 으로 데이터 셋을 나눠주자.

In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=4)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(7000, 5)
(7000,)
(3000, 5)
(3000,)


## Logistic Regression Model

이제 로지스틱 회귀 모델을 사용해서 유저들의 CTR을 예측해보자. 

In [29]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
Lr=LogisticRegression(C=0.01,random_state=0)

# 학습
Lr.fit(x_train,y_train)

# 추론
y_pred=Lr.predict(x_test)
print("추론 결과 : ",y_pred)

추론 결과 :  [1 1 1 ... 1 0 0]


추론 확률을 predict_proba 메서드로 확인해보자.

In [30]:
y_pred_proba=Lr.predict_proba(x_test)
print("클래스 추론 확률 : ", y_pred_proba)

클래스 추론 확률 :  [[0.25450752 0.74549248]
 [0.38997928 0.61002072]
 [0.49597281 0.50402719]
 ...
 [0.10962091 0.89037909]
 [0.74725049 0.25274951]
 [0.53100259 0.46899741]]


이제 accuracy_score를 통해 정확도를 한번 살펴보자.

In [31]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.6786666666666666


데이터가 적어서 그런지 정확도가 굉장히 낮다.

사실 accuracy_score보단 좀 더 좋은 평가지표를 생각해야한다. 정밀도와 재현율의 trade-off를 고려한 F1 score를 사용해보자.

In [35]:
from sklearn.metrics import f1_score
print(f1_score(y_test,y_pred))

0.6698630136986302


# 후기
---

논문을 읽으면서 로지스틱 회귀 예측에 대한 얘기를 읽어서.. 뭐지?? 이게 왜 사용되지?? 싶었다. 

그래서 인터넷을 검색해서 CTR이 어떤 느낌으로 돌아가는지 간단한 예제를 돌려보게 되었다. 이건 아주 간단한 예지만 ㅎㅎ..
그냥 feature들을 이용해서 회귀 예측을 하는 것이다. 여기서 로지스틱 회귀로 0과 1의 확률을 연속적인 값으로 출력할 수 있기 때문에, 0과 1이 discrete한 값이더라도 회귀 예측을 활용할 수 있다..

## 번외 

근데 갑자기 궁금해진게, softmax는 logistic 에서 발전한 것으로 알고 있다. 그럼 엄연히 숫자를 구분하는 문제는 마지막 단에 softmax layer를 사용하는데.. 이건 softmax를 사용했으니 회귀 예측이라고 볼 수 있는 걸까?? 싶네